In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import time
import os
import pandas as pd

d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()
def count_pos(arr):
  count = 0
  for num in arr:
    if num > 0:
        count += 1

  print("Number of positive elements in the array:", count)


In [5]:
def resizeImage(img_path, new_width):
    # Load the image
    img = cv2.imread(img_path)
    # Get the image dimensions
    height, width = img.shape[:2]
    # Calculate the aspect ratio
    ratio = float(new_width) / width
    # Calculate the new height
    new_height = int(height * ratio)
    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_img
    

def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    denominator = dx_dt**2 + dy_dt**2
    curvature = np.zeros_like(denominator)
    nonzero_denominator = denominator != 0
    curvature[nonzero_denominator] = (dx_dt[nonzero_denominator] * d2y_dt2[nonzero_denominator] - d2x_dt2[nonzero_denominator] * dy_dt[nonzero_denominator]) / denominator[nonzero_denominator]**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i, x, y, r, memo):
    # Check if the result has already been computed
    if (i, r) in memo:
        return memo[(i, r)]
    # Compute the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    # Memoize the result
    memo[(i, r)] = int_curv
    return int_curv



def get_area_measure(contour, i, radius, thresh,prev_center,prev_mask):
    # Compute current circle center and mask
    center = (contour[i][0][0], contour[i][0][1])
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, center, radius, (255, 255, 255), -1)
    
    # Compute mask difference to find changed pixels
    diff_mask = cv2.absdiff(circle_mask, prev_mask)
    changed_pixels = np.where(diff_mask > 0)
    
    # Compute intersection area using only changed pixels
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    changed_intersection_mask = intersection_mask[changed_pixels]
    intersection_area = np.sum(changed_intersection_mask) / 255
    
    # Update previous center and mask
    prev_center = center
    prev_mask = circle_mask
    
    # Compute area fraction
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area
    
    # Return area fraction
    return intersection_fraction


def get_curvatures(contours, radius_arr, thresh):
    curvature_arr = []
    # Loop over each contour
    for contour in contours:
        curr_1 = []
        curr_2 = []
        curr_area_arr = []
        curr_int_arr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        memo = {}
        for radius in radius_arr:
          prev_center = contour[0][0]
          prev_mask = np.zeros_like(gray)
          circle_perimeters = []

          for i in range(len(contour)):
                curr_area = get_area_measure(contour,i,radius,thresh,prev_center,prev_mask)
                curr_int = get_integral_curvature(i, x, y, radius, memo)
                curr_int_arr.append(curr_int)
                curr_area_arr.append(curr_area)
          curr_1.append(curr_int_arr)
          curr_2.append(curr_area_arr)

        curvature_arr.append(curr_1)
        curvature_arr.append(curr_2)

    return curvature_arr


In [11]:
# specify path to the folder containing image files
radius_arr = np.arange(10, 131, 5)
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"

counter = 0

# initialize empty lists for features and labels
hocs_area_list = []
hocs_int_list = []
labels = []
error_num = 0

# loop through all subfolders in the path
for foldername in os.listdir(path):
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue

    if counter == 1:
        break
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        try:
            
            start_time = time.time()
            # Load the image
            # resize the image
            img = resizeImage(filepath, 250)

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Find the contours in the image
            # Threshold the image to get a binary image
            ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

            # Find the contours in the image
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            radius_arr = np.arange(10, 131, 5)
            merged_contours = []

            for contour in contours:
                merged_contours += contour.tolist()
            oc = np.array([merged_contours])

            currvature_arr = get_curvatures(oc,radius_arr, thresh)
            curvature_arr = np.array(currvature_arr)

            area_hist_values = []
            for i in range(curvature_arr[1].shape[0]):
                counts, bin_edges = np.histogram(curvature_arr[1][i], bins=21) # adjust the number of bins as needed
                area_hist_values.append(counts)

            int_hist_values = []
            for i in range(curvature_arr[0].shape[0]):
                counts, bin_edges = np.histogram(curvature_arr[0][i], bins=21) # adjust the number of bins as needed
                int_hist_values.append(counts)


            area_data = np.array([np.array(hist) for hist in area_hist_values])
            int_data = np.array([np.array(hist) for hist in int_hist_values])

            area_hocs = area_data.flatten().tolist()
            int_hocs = int_data.flatten().tolist()

            hocs_area_list.append(area_hocs)
            hocs_int_list.append(int_hocs)
            labels.append(foldername)

            end_time = time.time()
            total_time = end_time - start_time

        except:
            error_num += 1
            print(error_num)
            pass

        
    print(f"Finished plant: {foldername}")
    
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"area_hist_values" : hocs_area_list, "int_hist_values" : hocs_int_list})
data["plant"] = labels
data.to_csv("lf_data.csv", index=False)

[675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119, 104, 93, 96, 83, 45, 23, 17, 11, 10, 13, 20, 675, 405, 285, 220, 288, 295, 238, 175, 160, 119,

KeyboardInterrupt: 

In [32]:
# Load the image
img = cv2.imread('13002228285126.png')
# set the new size
new_size = (250, 250)

# resize the image
img = cv2.resize(img, new_size)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image to get a binary image
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

blur = cv2.GaussianBlur(gray, (5,5), 0)

# Apply Canny edge detection to get the edges
edges = cv2.Canny(blur, 100, 200)

# Find the contours in the image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

radius_arr = np.arange(10, 131, 5)
merged_contours = []
for contour in contours:
    merged_contours += contour.tolist()
oc = np.array([merged_contours])

